In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import pickle as pkl

In [ ]:
class Simulation_o:
    def __init__(self, QAM = 4,Num_cc = np.arange(0,10), Num_ii = 100,
                 SNR_DB = np.arange(2,14,2), Nt = 16, Nr = 16, cesit = 0,
                 cesir = 0,channel_error =.0):
        self.Num_cc = Num_cc
        self.Num_ii = Num_ii
        self.SNR_DB = SNR_DB
        self.N_t = Nt
        self.N_r = Nr
        self.QAM = QAM
        self.cesit = cesit
        self.cesir = cesir
        self.channel_error = channel_error


    def data_generation(self):
        m=1
        M=[]
        for i in range(int(np.sqrt(self.QAM)/2)):
            M.append(m)
            M.append(-m)
            m+=2

        s = {}
        H = {}
        r = {}
        var = {}

        d =np.abs(M[0]-M[1])
        E = (self.QAM-1)*((d*d)/2)/3

        for cc in self.Num_cc:
            Z=1*self.N_r # RIS dimention element
            np.random.seed(cc)
            h = np.zeros((self.Num_ii,self.N_r,self.N_t)) + np.sqrt(0.5/self.N_r)*(np.random.randn(self.N_r,self.N_t)+np.random.randn(self.N_r,self.N_t)*1j)


            ####################  RIS ######################
            h1 = np.zeros((self.Num_ii,self.N_r,Z)) + np.sqrt(0.5/self.N_r)*(np.random.randn(self.N_r,Z)+np.random.randn(self.N_r,Z)*1j)
            h2 = np.zeros((self.Num_ii,Z,self.N_t)) + np.sqrt(0.5/self.N_r)*(np.random.randn(Z,self.N_t)+np.random.randn(Z,self.N_t)*1j)

            alfa = np.random.rand()
            teta_deg = np.random.rand(Z) * 180
            teta_gra = np.deg2rad(teta_deg)
            Teta = np.exp(1j * teta_gra)
            Phi = np.diag(Teta)

            hh = h1@(alfa*Phi)@h2
            h=h+hh
            # print(Phi)
            ####################  RIS ######################




            ####################  Channel estimation error ######################
            h_e = np.sqrt(0.5/self.N_r)*((np.random.randn(self.N_r,self.N_t)+np.random.randn(self.N_r,self.N_t)*1j))*np.sqrt(self.channel_error/2 )
            ####################  Channel estimation error ######################



            ####################  corolation ######################

            Rr = np.zeros((self.N_r, self.N_r))+0*1j
            Rt = np.zeros((self.N_t, self.N_t))+0*1j

            angle_deg = np.random.rand() * 90
            angle_rad = np.deg2rad(angle_deg)
            exponential_complex = np.exp(1j * angle_rad)

            rr = (self.cesir* exponential_complex) ** np.arange(self.N_r)
            tt = (self.cesit * exponential_complex) ** np.arange(self.N_t)

            for nn in range(self.N_t):
                Rt[nn, nn:self.N_t] = tt[:self.N_t-nn]

            for na in range(self.N_r):
                Rr[na, na:self.N_r] = rr[:self.N_r-na]

            Rt = Rt + np.conj(Rt.T) - np.eye(self.N_t)
            Rr = Rr + np.conj(Rr.T) - np.eye(self.N_r)

            ####################  corolation ######################

            h = Rr@h@Rt
            for snr in self.SNR_DB:
                X = np.random.choice(M,(self.Num_ii,self.N_t,1))+np.random.choice(M,(self.Num_ii,self.N_t,1))*1j
                x = X/np.sqrt(E)
                vv = np.sqrt(0.5/(10** (snr/10))) * (np.random.randn(self.Num_ii,self.N_r,1)+np.random.randn(self.Num_ii,self.N_r,1)*1j)

                y = h@x+vv

                if cc == self.Num_cc[0]:
                    var[snr] = vv
                    s[snr]=x
                    H[snr]=h+h_e
                    r[snr]=y
                else:
                    var[snr]=np.concatenate((var[snr],vv),0)
                    s[snr] = np.concatenate((s[snr],x),0)
                    H[snr] = np.concatenate((H[snr],h+h_e),0)
                    r[snr] = np.concatenate((r[snr],y),0)

        return s, H, r, var

In [ ]:
Num_cc = np.arange(0,100)
Num_ii = 100
SNR_DB = np.arange(10,24,2)
Nt = 8
Nr = 16
QAM = 16
cesit = .0
cesir = .0
channel_error_var=.0

data = Simulation_o(QAM = QAM,Num_cc = Num_cc, Num_ii = Num_ii,
                 SNR_DB = SNR_DB, Nt = Nt, Nr = Nr, cesit=cesit, cesir=cesir,channel_error =channel_error_var)
s, H, r, n = data.data_generation()


E = (QAM-1)*((2*2)/2)/3
E

# Convert complex Data to real value

In [ ]:
for snr in SNR_DB:
    sreal = np.real(s[snr])
    simag = np.imag(s[snr])
    s_real = np.concatenate((sreal,simag),1)

    rreal = np.real(r[snr])
    rimag = np.imag(r[snr])
    r_real = np.concatenate((rreal,rimag),1)

    hreal = np.real(H[snr])
    himag = np.imag(H[snr])
    hup = np.concatenate((hreal,-himag),2)
    hdown = np.concatenate((himag,hreal),2)
    h_real = np.concatenate((hup,hdown),1)

    nreal = np.real(n[snr])
    nimag = np.imag(n[snr])
    n_real = np.concatenate((nreal,nimag),1)

    if snr == SNR_DB[0]:

        s_ = s_real
        H_ = h_real
        r_ = r_real
        noise = n_real

    else:

        s_ = np.concatenate((s_ , s_real),0)
        H_ = np.concatenate((H_ , h_real),0)
        r_ = np.concatenate((r_ , r_real),0)
        noise = np.concatenate((noise , n_real),0)



‍# Make MMSE matrix

In [ ]:
sigmaI = (np.var(noise,1,keepdims=True)*np.eye(s_.shape[1]))

A = (np.transpose(H_,(0,2,1)) @ H_+sigmaI)
b = np.transpose(H_,(0,2,1)) @ r_

In [ ]:
L = np.tril(A)
U = np.triu(A)
D = -(A-U-L)
M_inv = np.linalg.inv(D) # In verse of diagonal value

In [ ]:
norm_A = np.mean(np.linalg.norm(A,ord=2,axis=(1,2))) # Avrage norm of MMSE matrix

In [ ]:
norm_A

# Seperate Testing and Training Dtata

In [ ]:
tr = np.arange(0,s_.shape[0],2)
te = np.arange(1,s_.shape[0],2)

s_tr = s_[tr]
s_te = s_[te]

H_tr = H_[tr]
H_te = H_[te]

A_tr = A[tr]
A_te = A[te]

b_tr = b[tr]
b_te = b[te]

M_i_tr = M_inv[tr]
M_i_te = M_inv[te]

In [ ]:
# EAch layer Architecture
class Custom_layer(keras.layers.Layer):
    def __init__(self , act):
        super(Custom_layer, self).__init__()

        self.act = act # Activation Function
        self.W1 = self.add_weight(initializer="normal", shape=(2*Nt,1),dtype="float32", trainable=True, name = 'W1_')# Gradient weight
        self.beta = self.add_weight(initializer="normal", shape=(1,1),dtype="float32", trainable=True, name = 'beta_')# First Momentum parameter
        self.xi = self.add_weight(initializer="normal", shape=(1,1),dtype="float32", trainable=True, name = 'xi_') #Second  Momentum parameter



    def call(self, inputs): # calling function for feeding forward
        x, A, b, x_p, x_pp, M_i = inputs[0], inputs[1], inputs[2], inputs[3], inputs[4], inputs[5] # Import data from pervius layer



        X = x + self.W1*M_i@(A@x-b)+self.beta*(x-x_p)+self.xi*(x-2*x_p+x_pp) # Operations of each layer

        return self.act(X)

# merge all layer
class Model_train(keras.Model):
    def __init__(self, model):
        super(Model_train, self).__init__()

        self.model = model # List of all layer

    def call(self, inputs):
        x, A, b, M_i= inputs[0], inputs[1], inputs[2], inputs[3]
        List = []

        X_p= x # output signal of 2 pervius layer
        X_pp=x # output signal of 3 pervius layer
        for i in range(len(self.model)):


            x_p=x

            x = self.model[i]([x, A, b,  X_p, X_pp,M_i]) # Feeding forward the considered layers

            X_pp=X_p
            X_p = x_p
            List.append(x)


        S = tf.concat(List,2)

        return S,x


In [ ]:
# making layer and set true value for loss function

def built_net(layer,s,Nt,act):
    model_list =[]
    S_list = []
    for i in range(layer):
        S_list.append(s)
        model_list.append(Custom_layer(act))

    return np.concatenate(S_list,2), model_list


# Activation function

def activation(x):
    x=np.sqrt(E)*x
    sig =1.5
    return (tf.tanh(sig*x)+tf.tanh(sig*x-2*sig)+tf.tanh(sig*x+2*sig))/np.sqrt(E)

# Loss function
def lf(outdv, Sd):
    outd=outdv[0]
    layer_ = Sd.shape[2]
    Sd = tf.constant(Sd,dtype=tf.float32)

    W = []
    Alpha = []
    Beta = []
    j=0
    for i in range (layer):

        W.append(Model.trainable_variables[j])
        j=j+1
        Beta.append(Model.trainable_variables[j])
        j=j+1
        Alpha.append(Model.trainable_variables[j])
        j=j+1



    lf1 = tf.reduce_sum(tf.abs(1-norm_A*tf.concat(W,1)-tf.concat(Beta,1)-tf.concat(Alpha,1))+
        tf.abs(tf.concat(Beta,1)+2*tf.concat(Alpha,1))+
        tf.abs(tf.concat(Alpha,1))-1)

    lf =   tf.norm(outd-Sd,ord=2,axis=1)**2


    return tf.math.reduce_sum(lf*np.log(np.arange(2,layer_+2)))+0.001*lf1


In [ ]:

len(SNR_DB)

In [ ]:
x= np.linspace(-2,2,100)
plt.plot(x,activation(x))
activation(.5);

In [ ]:
layer = 15 # number of layer

In [ ]:
S, model_list = built_net(layer,s_tr,Nt,activation) # Building layers

In [ ]:
Model = Model_train(model_list) # Merge layers

In [ ]:
lf(Model([s_tr*0, A_tr, b_tr, M_i_tr]),S).numpy() # Tetsing loss function

In [ ]:
lr =.1
opt = tf.keras.optimizers.Adam(lr) # call optimizer and set leraning rate


In [ ]:
# Training phase


epochs = 1000
for i in range(epochs):
    with tf.GradientTape() as tape:
        loss_value = lf(Model([s_tr*0, A_tr, b_tr, M_i_tr]), S)

    # Compute gradients
    gradients = tape.gradient(loss_value, Model.trainable_variables)

    # Apply gradients
    opt.apply_gradients(zip(gradients, Model.trainable_variables))

    # Print the loss for this epoch
    print('ep:', i, 'Loss= ', loss_value.numpy())

# Testing the testing data

In [ ]:
v= s_tr*0


s_out = Model([s_te*0, A_te, b_te, M_i_te, v])[1]


In [ ]:
M = np.array([-3,-1,1,3])
sym = np.array([[0,0],[0,1],[1,0],[1,1]])
S_Out = sym[np.expand_dims(np.argmin(np.abs(s_out*np.sqrt(E)-M),2),2)]
S_True = sym[np.expand_dims(np.argmin(np.abs(s_te*np.sqrt(E)-M),2),2)]

In [ ]:
er = S_Out != S_True
l = int(er.shape[0]/len(SNR_DB))

ber = []
for i in range(len(SNR_DB)):
    be = np.sum(er[i*l:(i+1)*l])/l/(4*Nt)

    ber.append(be)



In [ ]:
ber


In [ ]:
plt.semilogy(SNR_DB,ber)